## Urban Weather Generator (UWG) tutorial
---

[Urban Weather Generator](https://github.com/ladybug-tools/uwg/tree/master) (`uwg`) is a Python application for modeling the urban heat island effects. It is part of the [Ladybug Tools](https://github.com/ladybug-tools) collections, a *"collection of free and open-source computer applications that support environmentally-conscious design and simulation"*. Complete `uwg` API documentation can be found [here](https://www.ladybug.tools/uwg/docs/index.html). `uwg` works both as a CLI (Command Line Interface) and a Python scripting module! 

To import the module in a Python script or in your terminal (through Python or IPython), simply type this. You may also check if the installation was successful by running `uwg --help` in your terminal.
You can install `uwg` with `pip` right from your terminal. 

In [1]:
%pip install uwg
import uwg

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


#### Read EPW files in Python
---
While you guys use your own thermic simulation softwares, one might wonder how to handle EPW files in Python! Since EPW files are pretty much like .CSV files, we can treat them as such with `pandas`. To install `pandas`, run `pip install pandas` in your terminal. You could also use [pyepw](https://github.com/rbuffat/pyepw) or [pvlib](https://github.com/pvlib/pvlib-python) (`pvlib.iotools.epw`function), but I believe `pandas` leads to less errors. If you use R, you might want to check out this [module](https://hongyuanjia.github.io/eplusr/reference/Epw.html).

In [2]:
%pip install pandas
import pandas as pd


[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
def read_epw(file_path):
    # read EPW file as a list of lines
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    # extract EPW metadata 
    location = lines[0].strip()
    design_conditions = lines[1].strip()
    holidays_daylight_saving = lines[2].strip()
    comments_1 = lines[3].strip()
    comments_2 = lines[4].strip()
    # metadata dict
    metadata = {
        "location": location,
        "design_conditions": design_conditions,
        "holidays_daylight_saving": holidays_daylight_saving,
        "comments_1": comments_1,
        "comments_2": comments_2
    }

    # column names as given by the EPW file documentation
    epw_columns = [
        "year", "month", "day", "hour", "minute", "datasource", "dry_bulb_temperature", "dew_point_temperature",
        "relative_humidity", "atmospheric_pressure", "extraterrestrial_horizontal_radiation",
        "extraterrestrial_direct_normal_radiation", "horizontal_infrared_radiation_intensity_from_sky",
        "global_horizontal_radiation", "direct_normal_radiation", "diffuse_horizontal_radiation",
        "global_horizontal_illuminance", "direct_normal_illuminance", "diffuse_horizontal_illuminance",
        "zenith_luminance", "wind_direction", "wind_speed", "total_sky_cover", "opaque_sky_cover",
        "visibility", "ceiling_height", "present_weather_observation", "present_weather_codes",
        "precipitable_water", "aerosol_optical_depth", "snow_depth", "days_since_last_snowfall",
        "albedo", "liquid_precipitation_depth", "liquid_precipitation_quantity"
    ]

    # read weather data as csv, and skip metadata rows
    df = pd.read_csv(file_path, skiprows=8, header=None, names=epw_columns)

    return metadata, df

In [4]:
epw_file = "../resources/SGP_Singapore.486980_IWEC.epw"
metadata, epw_df = read_epw(epw_file)

In [5]:
print(epw_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 35 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   year                                              8760 non-null   int64  
 1   month                                             8760 non-null   int64  
 2   day                                               8760 non-null   int64  
 3   hour                                              8760 non-null   int64  
 4   minute                                            8760 non-null   int64  
 5   datasource                                        8760 non-null   object 
 6   dry_bulb_temperature                              8760 non-null   float64
 7   dew_point_temperature                             8760 non-null   float64
 8   relative_humidity                                 8760 non-null   int64  
 9   atmospheric_pressur

**Other option :** use [ladybug](https://github.com/ladybug-tools/ladybug), a library developped for weather data analysis and visualisation! 

In [10]:
%pip install ladybug-core
# load epw weather data
from ladybug.epw import EPW
epw_data = EPW(epw_file)
dry_bulb_temp = epw_data.dry_bulb_temperature
print(dry_bulb_temp)

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Hourly Continuous Data Collection
1/1 to 12/31 between 0 and 23 @1
Dry Bulb Temperature (C)
...8760 values...



[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### Use UWG
---
We'll work from the code provided in the repo's README. List of all parameters that can be included can be found [here](https://www.ladybug.tools/uwg/docs/uwg.uwg.html#uwg.uwg.UWG.B). While some of them have default values attributed to them, you may noticed that many parameters are by default not included in the calculations unless the user passes them as input.

**Examples:**
>*property* **h_temp**
>Get or set temperature height in meters.

>*property* **rurvegcover**
>Get or set fraction of rural ground covered by vegetation.

>*property* **vegstart**
>Get or set value from 1 to 12 for month at which vegetation starts to evapotranspire.
>This month corresponds to when the leaves of vegetation are assumed to be out.


In [7]:
from uwg import UWG

# Define the .epw, .uwg paths to create an uwg object.
epw_path = "../resources/SGP_Singapore.486980_IWEC.epw" # available in resources directory.

# Initialize the UWG model by passing parameters as arguments, or relying on defaults
model = UWG.from_param_args(epw_path=epw_path, bldheight=10, blddensity=0.5,
                            vertohor=0.8, grasscover=0.1, treecover=0.1, zone='1A')

# Uncomment these lines to initialize the UWG model using a .uwg parameter file
# param_path = "initialize_singapore.uwg"  # available in resources directory.
# model = UWG.from_param_file(param_path, epw_path=epw_path)

model.generate()
model.simulate()

# Write the simulation result to a file.
model.write_epw()

Simulating new temperature and humidity values for 31 days from 1/1.
Simulating Day 1
Simulating Day 2
Simulating Day 3
Simulating Day 4
Simulating Day 5
Simulating Day 6
Simulating Day 7
Simulating Day 8
Simulating Day 9
Simulating Day 10
Simulating Day 11
Simulating Day 12
Simulating Day 13
Simulating Day 14
Simulating Day 15
Simulating Day 16
Simulating Day 17
Simulating Day 18
Simulating Day 19
Simulating Day 20
Simulating Day 21
Simulating Day 22
Simulating Day 23
Simulating Day 24
Simulating Day 25
Simulating Day 26
Simulating Day 27
Simulating Day 28
Simulating Day 29
Simulating Day 30
Simulating Day 31
New climate file is generated at SGP_Singapore.486980_IWEC_UWG.epw.


In [8]:
epw_file = "SGP_Singapore.486980_IWEC_UWG.epw"
metadata, epw_df = read_epw(epw_file)

`UWG` is the main class provided by the `uwg` module. Parameters are described [here](https://www.ladybug.tools/uwg/docs/uwg.uwg.html). 
>*Morph a rural EPW file to urban conditions based on defined urban parameters.*

Other classes include: **BEM** (Building Energy Model), **RSM** (Rural Site Model), **UBL** (Urban Boundary Layer), **UCM** (Urban Canopy Model)...
